#### Method2 - KNN based tagging
##### Author : Kunal Kalwankar

In [ ]:
#imports and confing

import pandas as pd
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('/content/gdrive/MyDrive/cnn_dailymail')
!pip install Cython
import re
from pprint import pprint
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
#import tagged data

data = pd.read_csv('masterdata.csv')
print(data.head())

In [ ]:
# Function to remove extra blanks

def blankremover(x):
  x = [i for i in x if ''!=i]
  return x

# Preprocessing for text cleaning. Removal of syntax
data['article_list'] = data['article'].apply(lambda x: x.replace('..', "."))
data['article_list'] = data['article_list'].apply(lambda x: x.replace('.', " "))
data['article_list'] = data['article_list'].apply(lambda x:"".join(re.split(r'|\?|`|\‘|\’|#|\”|\}|\{|\)|\(|\]|\[|\"|\'|-|=|,|:|;', x.lower())))
data['article_list'] = data['article_list'].apply(lambda x:re.split(r'\s', x.lower()))
data['article_list'] = data['article_list'].apply(lambda x:blankremover(x))
data_list = data['article_list'].apply(lambda x:" ".join(x))

In [ ]:
# TFIDF Creation process

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_list)
print(X.shape)
test = list(vectorizer.get_feature_names_out())

In [ ]:
#SVD Creation Process
from sklearn.decomposition import  TruncatedSVD

svd =  TruncatedSVD(1000)
svd.fit(X)
print(svd.explained_variance_ratio_.sum())
print(svd.singular_values_)
tns = svd.transform(X)

print(tns.shape)

In [ ]:
# Kmeans cluster generation techinque

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0).fit(tns)
print(kmeans.labels_)
print(kmeans.cluster_centers_)
m = list(kmeans.predict(tns))

In [ ]:
#save the cluster tags for further usage
pd.DataFrame({'classname':m}).to_csv('KNN3.csv')

In [ ]:
#elbow plot generation functions

from matplotlib import pyplot as plt
import seaborn as sns

def elbow_plot(df):

    sse = {}
    
    for k in range(2, 15):
        print('KNN - K value: ',k)
        kmeans = KMeans(n_clusters=k, random_state=1)
        kmeans.fit(df)
        sse[k] = kmeans.inertia_
    
    plt.title('Elbow plot for K selection')
    plt.xlabel('k')
    plt.ylabel('SSE')
    sns.pointplot(x=list(sse.keys()),
                 y=list(sse.values()))
    plt.show()

#display elbowplot
elbow_plot(tns)